## DATA CLEANING & STRUCTURE VALIDATION

In [1]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-L3GBMQ5\\SQLEXPRESS;"
    "Database=Q4 Multi-Channel Performance & Forecast Alignment Analysis;"
    "Trusted_Connection=yes;"
)


---------------------

### Convert hourly performance -> daily

In [2]:
df_perf = pd.read_sql_query("SELECT * FROM Performance", conn)

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_2920\2164272931.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_perf = pd.read_sql_query("SELECT * FROM Performance", conn)


In [3]:
df_perf.head(10)

,date,channel_id,campaign_id,spend,impressions,clicks,conversions,revenue,POAS,CTR,CVR,performance_id
0,2025-01-01,1,1001,191.389999,4738,512,22,1222.260010,6.390000,0.1081,0.0430,1
1,2025-01-01,1,1002,219.229996,2155,185,20,1806.890015,8.240000,0.0858,0.1081,2
2,2025-01-01,1,1003,619.710022,10998,406,10,927.969971,1.500000,0.0369,0.0246,3
3,2025-01-01,1,1004,398.059998,12398,1141,108,6762.640137,16.990000,0.0920,0.0947,4
4,2025-01-01,1,1005,268.309998,10595,429,52,2109.110107,7.860000,0.0405,0.1212,5
5,2025-01-01,1,1006,550.659973,6146,434,74,2583.179932,4.690000,0.0706,0.1705,6
6,2025-01-01,1,1007,447.760010,2968,106,10,889.969971,1.990000,0.0357,0.0943,7
7,2025-01-01,1,1008,733.450012,12289,215,17,1078.359985,1.470000,0.0175,0.0791,8
8,2025-01-01,1,1009,24.020000,1828,114,17,804.010010,33.470001,0.0624,0.1491,9
9,2025-01-01,1,1010,75.010002,5888,397,50,3660.219971,48.799999,0.0674,0.1259,10


In [4]:
# convert date
df_perf['Date'] = pd.to_datetime(df_perf['date']).dt.date

df_daily = (df_perf
            .groupby(['Date', 'channel_id', 'campaign_id'])
            .agg({'impressions': 'sum'
                  ,'clicks': 'sum'
                  ,'conversions': 'sum'
                  ,'spend': 'sum'
                  ,'revenue': 'sum'
})
            .reset_index()
            )

In [5]:
# KPIs
df_daily['CTR'] = df_daily['clicks'] / df_daily['impressions']
df_daily['CVR'] = df_daily['conversions'] / df_daily['clicks']
df_daily['POAS'] = df_daily['revenue'] / df_daily['spend']

In [6]:
df_perf.head()

,date,channel_id,campaign_id,spend,impressions,clicks,conversions,revenue,POAS,CTR,CVR,performance_id,Date
0,2025-01-01,1,1001,191.389999,4738,512,22,1222.260010,6.39,0.1081,0.0430,1,2025-01-01
1,2025-01-01,1,1002,219.229996,2155,185,20,1806.890015,8.24,0.0858,0.1081,2,2025-01-01
2,2025-01-01,1,1003,619.710022,10998,406,10,927.969971,1.50,0.0369,0.0246,3,2025-01-01
3,2025-01-01,1,1004,398.059998,12398,1141,108,6762.640137,16.99,0.0920,0.0947,4,2025-01-01
4,2025-01-01,1,1005,268.309998,10595,429,52,2109.110107,7.86,0.0405,0.1212,5,2025-01-01


-----------------

### product_impact cleaning

In [7]:
df_imp = pd.read_sql_query("SELECT * FROM product_impact", conn)

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_2920\904220287.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_imp = pd.read_sql_query("SELECT * FROM product_impact", conn)


In [8]:
df_imp.head(10)

,date,channel_id,awareness_index,consideration_index,impact_score,impact_id,performance_id
0,2025-01-01,1,0.8447,0.7692,1.6891,1,None
1,2025-01-01,2,0.1412,0.1350,1.8464,2,None
2,2025-01-01,3,0.1052,0.0609,1.1839,3,None
3,2025-01-01,4,0.2765,0.2284,1.9436,4,None
4,2025-01-01,5,0.5189,0.2093,0.9221,5,None
5,2025-01-02,1,0.1175,0.0763,1.8520,6,None
6,2025-01-02,2,0.5294,0.1068,0.9285,7,None
7,2025-01-02,3,0.8706,0.6398,1.4252,8,None
8,2025-01-02,4,0.8735,0.4647,0.8139,9,None
9,2025-01-02,5,0.7808,0.4576,0.5635,10,None


In [9]:
df_imp['date'] = pd.to_datetime(df_imp['date']).dt.date

In [10]:
df_imp[['awareness_index','consideration_index','impact_score']] = \
    df_imp[['awareness_index','consideration_index','impact_score']].fillna(0)

In [11]:
df_imp.head()

,date,channel_id,awareness_index,consideration_index,impact_score,impact_id,performance_id
0,2025-01-01,1,0.8447,0.7692,1.6891,1,None
1,2025-01-01,2,0.1412,0.1350,1.8464,2,None
2,2025-01-01,3,0.1052,0.0609,1.1839,3,None
3,2025-01-01,4,0.2765,0.2284,1.9436,4,None
4,2025-01-01,5,0.5189,0.2093,0.9221,5,None


-----------------------------

### Join daily_performance + product_impact + forecasting

In [12]:
df_fore = pd.read_sql("SELECT * FROM forecasting", conn)

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_2920\1651432634.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fore = pd.read_sql("SELECT * FROM forecasting", conn)


In [13]:
df_fore.head(10)

,date,channel_id,campaign_id,forecast_impressions,forecast_clicks,forecast_conversions,forecast_revenue,actual_impressions,actual_clicks,actual_conversions,actual_revenue,forecast_id,performance_id
0,2025-01-01,1,1001,4919,481,71,5347.939941,5260,80,9,566.140015,1,None
1,2025-01-01,1,1002,1959,27,2,98.650002,1820,64,7,822.429993,2,None
2,2025-01-01,1,1003,4585,96,10,236.869995,4637,141,12,1130.400024,3,None
3,2025-01-01,1,1004,1588,51,1,99.669998,1658,23,1,101.410004,4,None
4,2025-01-01,1,1005,2797,264,27,1876.910034,2404,159,12,961.390015,5,None
5,2025-01-01,1,1006,1137,106,9,593.460022,1054,50,5,402.910004,6,None
6,2025-01-01,1,1007,918,27,2,225.979996,885,46,2,157.479996,7,None
7,2025-01-01,1,1008,2954,253,25,2524.669922,3117,140,11,449.660004,8,None
8,2025-01-01,1,1009,3437,207,19,1843.010010,3550,176,10,324.630005,9,None
9,2025-01-01,1,1010,3905,169,4,207.309998,3918,276,37,1050.589966,10,None


In [14]:
df_fore['date'] = pd.to_datetime(df_fore['date']).dt.date

In [15]:
df_daily.rename(columns={'Date': 'date'}, inplace=True)

In [16]:
df_master = df_daily.merge(df_imp, on=['date', 'channel_id'], how='left')

### Logical Validation Flags

In [18]:
df_master['flag_clicks_gt_impressions'] = (df_master['clicks'] > df_master['impressions']).astype(int)
df_master['flag_conversions_gt_clicks'] = (df_master['conversions'] > df_master['clicks']).astype(int)
df_master['flag_negative_spend'] = (df_master['spend'] < 0).astype(int)

### Save Clean master_daily

In [21]:
cursor = conn.cursor()

insert_query = """
INSERT INTO master_daily (
    date, channel_id, campaign_id,
    spend, impressions, clicks, conversions, revenue,
    POAS, CTR, CVR,
    awareness_index, consideration_index, impact_score,
    forecasted_revenue, expected_conversions,
    flag_clicks_gt_impressions, flag_conversions_gt_clicks, flag_negative_spend
)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

for _, row in df_master.iterrows():
    cursor.execute(insert_query, tuple(row))

conn.commit()
